# 06 keras callbacks optimization
**Location: TensorVerseHub/notebooks/02_neural_networks_with_keras/06_keras_callbacks_optimization.ipynb**

In [ ]:
import tensorflow as tf
import numpy as np
print(f"TensorFlow version: {tf.__version__}")

# tf.keras Training Optimization & Advanced Techniques

**File Location:** `notebooks/02_neural_networks_with_keras/06_training_optimization_keras.ipynb`

Master advanced training optimization techniques in tf.keras including callbacks, optimizers, learning rate schedules, regularization methods, and performance monitoring. Build production-ready training pipelines with automated hyperparameter tuning and advanced optimization strategies.

## Learning Objectives
- Master tf.keras callbacks for training automation and monitoring
- Implement advanced optimizers and learning rate scheduling
- Apply comprehensive regularization techniques
- Build automated hyperparameter tuning systems
- Optimize training performance and memory usage
- Monitor and visualize training dynamics

---

## 1. Advanced tf.keras Callbacks

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification, load_digits
from sklearn.model_selection import train_test_split
import os
import datetime
import json
import warnings
warnings.filterwarnings('ignore')

print(f"TensorFlow version: {tf.__version__}")
tf.random.set_seed(42)
np.random.seed(42)

# Setup logging directories
log_base_dir = os.path.join("logs", "training_optimization", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
os.makedirs(log_base_dir, exist_ok=True)

# Create comprehensive dataset
def create_datasets():
    """Create various datasets for optimization experiments"""
    
    # Large classification dataset
    X_large, y_large = make_classification(
        n_samples=10000, n_features=50, n_classes=5, 
        n_redundant=5, n_informative=30, random_state=42
    )
    
    # Digits dataset for computer vision
    digits = load_digits()
    X_digits, y_digits = digits.data, digits.target
    
    # Split datasets
    X_train, X_test, y_train, y_test = train_test_split(
        X_large.astype(np.float32), y_large, test_size=0.2, random_state=42
    )
    
    X_digits_train, X_digits_test, y_digits_train, y_digits_test = train_test_split(
        X_digits.astype(np.float32), y_digits, test_size=0.2, random_state=42
    )
    
    return (X_train, X_test, y_train, y_test), (X_digits_train, X_digits_test, y_digits_train, y_digits_test)

(X_train, X_test, y_train, y_test), (X_digits_train, X_digits_test, y_digits_train, y_digits_test) = create_datasets()

print(f"Main dataset: {X_train.shape}, {X_test.shape}")
print(f"Digits dataset: {X_digits_train.shape}, {X_digits_test.shape}")

# Advanced callback implementations
class LearningRateLogger(tf.keras.callbacks.Callback):
    """Log learning rate changes with visualization"""
    
    def __init__(self, log_dir):
        super().__init__()
        self.log_dir = log_dir
        self.writer = tf.summary.create_file_writer(os.path.join(log_dir, 'lr_logs'))
        
    def on_epoch_end(self, epoch, logs=None):
        current_lr = tf.keras.backend.get_value(self.model.optimizer.learning_rate)
        
        with self.writer.as_default():
            tf.summary.scalar('learning_rate', current_lr, step=epoch)
        
        if epoch % 10 == 0:
            print(f"Epoch {epoch}: Learning Rate = {current_lr:.6f}")

class GradientNormCallback(tf.keras.callbacks.Callback):
    """Monitor gradient norms during training"""
    
    def __init__(self, log_dir, log_freq=10):
        super().__init__()
        self.log_dir = log_dir
        self.log_freq = log_freq
        self.writer = tf.summary.create_file_writer(os.path.join(log_dir, 'gradient_logs'))
        self.batch_count = 0
        
    def on_batch_end(self, batch, logs=None):
        self.batch_count += 1
        
        if self.batch_count % self.log_freq == 0:
            # Get gradients (simplified approach)
            with tf.GradientTape() as tape:
                # Use a small batch for gradient calculation
                sample_x = self.validation_data[0][:32] if self.validation_data else None
                sample_y = self.validation_data[1][:32] if self.validation_data else None
                
                if sample_x is not None:
                    predictions = self.model(sample_x, training=True)
                    loss = self.model.compiled_loss(sample_y, predictions)
                    
                    gradients = tape.gradient(loss, self.model.trainable_variables)
                    
                    # Calculate gradient norms
                    grad_norms = []
                    for grad in gradients:
                        if grad is not None:
                            grad_norms.append(tf.norm(grad))
                    
                    if grad_norms:
                        total_grad_norm = tf.norm(grad_norms)
                        max_grad_norm = tf.reduce_max(grad_norms)
                        
                        with self.writer.as_default():
                            tf.summary.scalar('total_gradient_norm', total_grad_norm, step=self.batch_count)
                            tf.summary.scalar('max_gradient_norm', max_grad_norm, step=self.batch_count)

class AdaptiveEarlyStopping(tf.keras.callbacks.Callback):
    """Adaptive early stopping with patience adjustment"""
    
    def __init__(self, monitor='val_loss', patience=10, min_delta=0.001, 
                 restore_best_weights=True, adaptive_patience=True):
        super().__init__()
        self.monitor = monitor
        self.patience = patience
        self.original_patience = patience
        self.min_delta = min_delta
        self.restore_best_weights = restore_best_weights
        self.adaptive_patience = adaptive_patience
        
        self.wait = 0
        self.stopped_epoch = 0
        self.best = None
        self.best_weights = None
        self.improvement_history = []
        
    def on_train_begin(self, logs=None):
        self.wait = 0
        self.stopped_epoch = 0
        self.best = np.Inf if 'loss' in self.monitor else -np.Inf
        
    def on_epoch_end(self, epoch, logs=None):
        current = logs.get(self.monitor)
        if current is None:
            return
            
        # Check for improvement
        if 'loss' in self.monitor:
            improved = current < self.best - self.min_delta
        else:
            improved = current > self.best + self.min_delta
            
        if improved:
            self.best = current
            self.wait = 0
            if self.restore_best_weights:
                self.best_weights = self.model.get_weights()
            
            # Track improvement
            self.improvement_history.append(epoch)
            
            # Adaptive patience adjustment
            if self.adaptive_patience and len(self.improvement_history) > 5:
                recent_improvements = np.diff(self.improvement_history[-5:])
                avg_improvement_gap = np.mean(recent_improvements)
                
                # Adjust patience based on improvement frequency
                if avg_improvement_gap < 5:  # Frequent improvements
                    self.patience = min(self.original_patience * 2, 50)
                elif avg_improvement_gap > 15:  # Infrequent improvements
                    self.patience = max(self.original_patience // 2, 5)
                    
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                if self.restore_best_weights and self.best_weights:
                    self.model.set_weights(self.best_weights)
                    
    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0:
            print(f"\nEarly stopping at epoch {self.stopped_epoch + 1}")
            print(f"Best {self.monitor}: {self.best:.4f}")

class PerformanceProfiler(tf.keras.callbacks.Callback):
    """Profile training performance and resource usage"""
    
    def __init__(self, log_dir):
        super().__init__()
        self.log_dir = log_dir
        self.epoch_times = []
        self.batch_times = []
        self.memory_usage = []
        
    def on_train_begin(self, logs=None):
        import psutil
        self.process = psutil.Process()
        
    def on_epoch_begin(self, epoch, logs=None):
        import time
        self.epoch_start_time = time.time()
        
    def on_epoch_end(self, epoch, logs=None):
        import time
        epoch_time = time.time() - self.epoch_start_time
        self.epoch_times.append(epoch_time)
        
        # Memory usage
        memory_mb = self.process.memory_info().rss / 1024 / 1024
        self.memory_usage.append(memory_mb)
        
        if epoch % 5 == 0:
            print(f"Epoch {epoch}: {epoch_time:.2f}s, Memory: {memory_mb:.1f}MB")
            
    def on_batch_begin(self, batch, logs=None):
        import time
        self.batch_start_time = time.time()
        
    def on_batch_end(self, batch, logs=None):
        import time
        batch_time = time.time() - self.batch_start_time
        self.batch_times.append(batch_time)
        
    def on_train_end(self, logs=None):
        # Save performance report
        report = {
            'avg_epoch_time': np.mean(self.epoch_times),
            'total_training_time': np.sum(self.epoch_times),
            'avg_batch_time': np.mean(self.batch_times),
            'peak_memory_mb': np.max(self.memory_usage),
            'avg_memory_mb': np.mean(self.memory_usage)
        }
        
        report_path = os.path.join(self.log_dir, 'performance_report.json')
        with open(report_path, 'w') as f:
            json.dump(report, f, indent=2)
            
        print(f"\nPerformance Report:")
        print(f"  Average epoch time: {report['avg_epoch_time']:.2f}s")
        print(f"  Total training time: {report['total_training_time']:.2f}s")
        print(f"  Peak memory usage: {report['peak_memory_mb']:.1f}MB")

# Test advanced callbacks
def test_advanced_callbacks():
    """Test advanced callback implementations"""
    
    print("=== Testing Advanced Callbacks ===")
    
    # Create model
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(5, activation='softmax')
    ])
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    # Setup callbacks
    callbacks = [
        LearningRateLogger(log_base_dir),
        AdaptiveEarlyStopping(monitor='val_loss', patience=8, adaptive_patience=True),
        PerformanceProfiler(log_base_dir),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1
        ),
        tf.keras.callbacks.ModelCheckpoint(
            os.path.join(log_base_dir, 'best_model.h5'),
            save_best_only=True, monitor='val_accuracy', mode='max'
        )
    ]
    
    # Train model
    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=50,
        batch_size=64,
        callbacks=callbacks,
        verbose=1
    )
    
    return model, history

# Run callback tests
callback_model, callback_history = test_advanced_callbacks()

## 2. Advanced Optimizers and Learning Rate Schedules

In [ ]:
# Custom learning rate schedules
class WarmupCosineDecaySchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    """Warmup followed by cosine decay schedule"""
    
    def __init__(self, initial_learning_rate, decay_steps, warmup_steps=1000, alpha=0.0):
        super().__init__()
        self.initial_learning_rate = initial_learning_rate
        self.decay_steps = decay_steps
        self.warmup_steps = warmup_steps
        self.alpha = alpha
        
    def __call__(self, step):
        # Warmup phase
        warmup_lr = self.initial_learning_rate * (step / self.warmup_steps)
        
        # Cosine decay phase
        cosine_decay_lr = self.alpha + (self.initial_learning_rate - self.alpha) * (
            1 + tf.cos(tf.constant(np.pi) * step / self.decay_steps)
        ) / 2
        
        return tf.cond(
            step < self.warmup_steps,
            lambda: warmup_lr,
            lambda: cosine_decay_lr
        )
    
    def get_config(self):
        return {
            'initial_learning_rate': self.initial_learning_rate,
            'decay_steps': self.decay_steps,
            'warmup_steps': self.warmup_steps,
            'alpha': self.alpha
        }

class CyclicalLearningRate(tf.keras.optimizers.schedules.LearningRateSchedule):
    """Cyclical learning rate with triangular policy"""
    
    def __init__(self, base_lr, max_lr, step_size, mode='triangular'):
        super().__init__()
        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        
    def __call__(self, step):
        cycle = tf.floor(1 + step / (2 * self.step_size))
        x = tf.abs(step / self.step_size - 2 * cycle + 1)
        
        if self.mode == 'triangular':
            scale_factor = 1.0
        elif self.mode == 'triangular2':
            scale_factor = 1.0 / (2.0 ** (cycle - 1))
        else:  # exp_range
            scale_factor = 0.99999 ** step
            
        lr = self.base_lr + (self.max_lr - self.base_lr) * tf.maximum(0.0, 1 - x) * scale_factor
        return lr
    
    def get_config(self):
        return {
            'base_lr': self.base_lr,
            'max_lr': self.max_lr,
            'step_size': self.step_size,
            'mode': self.mode
        }

class OneCycleLR(tf.keras.optimizers.schedules.LearningRateSchedule):
    """One cycle learning rate policy"""
    
    def __init__(self, max_lr, total_steps, div_factor=25, final_div_factor=10000):
        super().__init__()
        self.max_lr = max_lr
        self.total_steps = total_steps
        self.div_factor = div_factor
        self.final_div_factor = final_div_factor
        
        self.initial_lr = max_lr / div_factor
        self.final_lr = self.initial_lr / final_div_factor
        
    def __call__(self, step):
        pct = step / self.total_steps
        
        # First half: increase to max_lr
        if pct <= 0.5:
            lr = self.initial_lr + (self.max_lr - self.initial_lr) * pct * 2
        # Second half: decrease to final_lr
        else:
            lr = self.max_lr - (self.max_lr - self.final_lr) * (pct - 0.5) * 2
            
        return tf.maximum(lr, self.final_lr)
    
    def get_config(self):
        return {
            'max_lr': self.max_lr,
            'total_steps': self.total_steps,
            'div_factor': self.div_factor,
            'final_div_factor': self.final_div_factor
        }

# Custom optimizer with gradient clipping
class AdamWWithClipping(tf.keras.optimizers.Optimizer):
    """Adam with weight decay and gradient clipping"""
    
    def __init__(self, learning_rate=0.001, beta_1=0.9, beta_2=0.999, 
                 weight_decay=0.01, clipnorm=1.0, epsilon=1e-7, name="AdamWClip", **kwargs):
        super().__init__(name, **kwargs)
        
        self._set_hyper('learning_rate', kwargs.get('lr', learning_rate))
        self._set_hyper('beta_1', beta_1)
        self._set_hyper('beta_2', beta_2)
        self._set_hyper('weight_decay', weight_decay)
        self._set_hyper('clipnorm', clipnorm)
        self.epsilon = epsilon or tf.keras.backend.epsilon()
        
    def _create_slots(self, var_list):
        for var in var_list:
            self.add_slot(var, 'm')  # First moment
            self.add_slot(var, 'v')  # Second moment
            
    def _resource_apply_dense(self, grad, var):
        var_dtype = var.dtype.base_dtype
        lr_t = self._decayed_lr(var_dtype)
        
        # Get hyperparameters
        beta_1_t = self._get_hyper('beta_1', var_dtype)
        beta_2_t = self._get_hyper('beta_2', var_dtype)
        weight_decay_t = self._get_hyper('weight_decay', var_dtype)
        clipnorm_t = self._get_hyper('clipnorm', var_dtype)
        
        # Clip gradients
        if clipnorm_t > 0:
            grad = tf.clip_by_norm(grad, clipnorm_t)
        
        # Get slots
        m = self.get_slot(var, 'm')
        v = self.get_slot(var, 'v')
        
        # Update biased first moment estimate
        m_t = beta_1_t * m + (1.0 - beta_1_t) * grad
        
        # Update biased second raw moment estimate
        v_t = beta_2_t * v + (1.0 - beta_2_t) * tf.square(grad)
        
        # Bias correction
        m_corr = m_t / (1.0 - tf.pow(beta_1_t, tf.cast(self.iterations + 1, var_dtype)))
        v_corr = v_t / (1.0 - tf.pow(beta_2_t, tf.cast(self.iterations + 1, var_dtype)))
        
        # Update parameters
        update = lr_t * m_corr / (tf.sqrt(v_corr) + self.epsilon)
        
        # Add weight decay
        if weight_decay_t > 0:
            update += weight_decay_t * lr_t * var
            
        var_update = var.assign_sub(update)
        
        # Update slots
        m_update = m.assign(m_t)
        v_update = v.assign(v_t)
        
        return tf.group(var_update, m_update, v_update)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            'learning_rate': self._serialize_hyperparameter('learning_rate'),
            'beta_1': self._serialize_hyperparameter('beta_1'),
            'beta_2': self._serialize_hyperparameter('beta_2'),
            'weight_decay': self._serialize_hyperparameter('weight_decay'),
            'clipnorm': self._serialize_hyperparameter('clipnorm'),
            'epsilon': self.epsilon,
        })
        return config

# Test different optimizers and schedules
def compare_optimizers_and_schedules():
    """Compare different optimizer and learning rate schedule combinations"""
    
    print("\n=== Optimizer and Schedule Comparison ===")
    
    # Define configurations to test
    configs = {
        'Adam_Standard': {
            'optimizer': tf.keras.optimizers.Adam(learning_rate=0.001),
            'schedule': None
        },
        'Adam_CosineDecay': {
            'optimizer': tf.keras.optimizers.Adam,
            'schedule': tf.keras.optimizers.schedules.CosineDecay(0.001, 1000)
        },
        'Adam_WarmupCosine': {
            'optimizer': tf.keras.optimizers.Adam,
            'schedule': WarmupCosineDecaySchedule(0.001, 1000, 100)
        },
        'AdamW_OneCycle': {
            'optimizer': tf.keras.optimizers.AdamW,
            'schedule': OneCycleLR(0.003, 1000)
        },
        'Custom_AdamW_Clip': {
            'optimizer': AdamWWithClipping,
            'schedule': CyclicalLearningRate(0.0001, 0.001, 200)
        }
    }
    
    results = {}
    
    for name, config in configs.items():
        print(f"\nTesting {name}...")
        
        # Create model
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dense(5, activation='softmax')
        ])
        
        # Setup optimizer
        if config['schedule']:
            optimizer = config['optimizer'](learning_rate=config['schedule'])
        else:
            optimizer = config['optimizer']
        
        model.compile(
            optimizer=optimizer,
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        
        # Train with early stopping
        history = model.fit(
            X_train, y_train,
            validation_data=(X_test, y_test),
            epochs=30,
            batch_size=64,
            verbose=0,
            callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]
        )
        
        # Store results
        results[name] = {
            'final_train_acc': history.history['accuracy'][-1],
            'final_val_acc': history.history['val_accuracy'][-1],
            'best_val_acc': max(history.history['val_accuracy']),
            'epochs_trained': len(history.history['accuracy']),
            'final_loss': history.history['val_loss'][-1]
        }
        
        print(f"  Best Val Acc: {results[name]['best_val_acc']:.4f}")
        print(f"  Epochs: {results[name]['epochs_trained']}")
    
    # Print comparison
    print("\n=== Optimization Results Summary ===")
    print(f"{'Configuration':<20} {'Best Val Acc':<12} {'Final Val Acc':<12} {'Epochs':<8} {'Final Loss':<12}")
    print("-" * 80)
    
    for name, result in results.items():
        print(f"{name:<20} {result['best_val_acc']:<12.4f} "
              f"{result['final_val_acc']:<12.4f} {result['epochs_trained']:<8} "
              f"{result['final_loss']:<12.4f}")
    
    return results

# Run optimizer comparison
optimizer_results = compare_optimizers_and_schedules()

## 3. Comprehensive Regularization Techniques

In [ ]:
# Advanced regularization methods
class MixupCallback(tf.keras.callbacks.Callback):
    """Mixup data augmentation callback"""
    
    def __init__(self, alpha=0.2):
        super().__init__()
        self.alpha = alpha
        
    def on_batch_begin(self, batch, logs=None):
        # Get batch data (this is simplified - in practice you'd need to modify the data pipeline)
        pass  # Implementation would require custom training loop
        
class CutMixCallback(tf.keras.callbacks.Callback):
    """CutMix data augmentation callback"""
    
    def __init__(self, alpha=1.0):
        super().__init__()
        self.alpha = alpha

class DropBlock(tf.keras.layers.Layer):
    """DropBlock regularization layer"""
    
    def __init__(self, drop_rate=0.1, block_size=7, **kwargs):
        super().__init__(**kwargs)
        self.drop_rate = drop_rate
        self.block_size = block_size
        
    def call(self, inputs, training=None):
        if not training:
            return inputs
            
        # For 2D inputs, treat as flattened feature maps
        batch_size = tf.shape(inputs)[0]
        feature_size = tf.shape(inputs)[1]
        
        # Calculate gamma (DropBlock paper)
        gamma = self.drop_rate / (self.block_size ** 2)
        
        # Sample mask
        mask_shape = (batch_size, feature_size)
        mask = tf.random.uniform(mask_shape) < gamma
        
        # Apply block structure (simplified for 1D case)
        block_mask = tf.cast(mask, tf.float32)
        
        # Scale to maintain expected value
        scale = tf.cast(tf.size(block_mask), tf.float32) / tf.reduce_sum(block_mask)
        
        return inputs * block_mask * scale
        
    def get_config(self):
        config = super().get_config()
        config.update({
            'drop_rate': self.drop_rate,
            'block_size': self.block_size
        })
        return config

class SpectralNormalization(tf.keras.layers.Wrapper):
    """Spectral normalization wrapper for any layer"""
    
    def __init__(self, layer, power_iterations=1, **kwargs):
        super().__init__(layer, **kwargs)
        self.power_iterations = power_iterations
        
    def build(self, input_shape):
        super().build(input_shape)
        
        # Get the layer's kernel
        if hasattr(self.layer, 'kernel'):
            kernel = self.layer.kernel
            kernel_shape = kernel.shape
            
            # Create u and v vectors for power iteration
            self.u = self.add_weight(
                shape=(1, kernel_shape[-1]),
                initializer='random_normal',
                trainable=False,
                name='u_vector'
            )
            
    def call(self, inputs, training=None, **kwargs):
        if hasattr(self.layer, 'kernel') and training:
            # Power iteration to estimate spectral norm
            kernel = self.layer.kernel
            w_reshaped = tf.reshape(kernel, [-1, kernel.shape[-1]])
            
            u = self.u
            for _ in range(self.power_iterations):
                v = tf.nn.l2_normalize(tf.matmul(u, w_reshaped, transpose_b=True))
                u = tf.nn.l2_normalize(tf.matmul(v, w_reshaped))
            
            # Calculate spectral norm
            sigma = tf.matmul(tf.matmul(v, w_reshaped), u, transpose_b=True)
            
            # Normalize kernel
            kernel_normalized = kernel / sigma
            
            # Temporarily replace kernel
            original_kernel = self.layer.kernel
            self.layer.kernel = kernel_normalized
            
            # Update u vector
            self.u.assign(u)
            
            # Forward pass
            output = self.layer(inputs, **kwargs)
            
            # Restore original kernel
            self.layer.kernel = original_kernel
            
            return output
        else:
            return self.layer(inputs, **kwargs)

class StochasticDepth(tf.keras.layers.Layer):
    """Stochastic depth regularization"""
    
    def __init__(self, drop_prob=0.1, **kwargs):
        super().__init__(**kwargs)
        self.drop_prob = drop_prob
        
    def call(self, inputs, training=None):
        if not training:
            return inputs
            
        # Randomly drop the entire layer
        random_tensor = tf.random.uniform([tf.shape(inputs)[0], 1, 1])
        keep_prob = 1.0 - self.drop_prob
        
        # Create binary mask
        binary_tensor = tf.cast(random_tensor < keep_prob, tf.float32)
        
        # Scale output to maintain expected value
        output = inputs * binary_tensor / keep_prob
        
        return output
        
    def get_config(self):
        config = super().get_config()
        config.update({'drop_prob': self.drop_prob})
        return config

# Advanced regularized model
def create_heavily_regularized_model(input_shape, num_classes):
    """Create model with comprehensive regularization"""
    
    inputs = tf.keras.layers.Input(shape=input_shape)
    
    # Input normalization and noise
    x = tf.keras.layers.BatchNormalization()(inputs)
    x = tf.keras.layers.GaussianNoise(0.1)(x)
    
    # First block with multiple regularization techniques
    x = SpectralNormalization(tf.keras.layers.Dense(128, activation='relu'))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = DropBlock(drop_rate=0.1)(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    
    # Residual connection with stochastic depth
    residual = x
    x = SpectralNormalization(tf.keras.layers.Dense(128, activation='relu', 
                                                   kernel_regularizer=tf.keras.regularizers.l1_l2(0.01, 0.01)))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = StochasticDepth(drop_prob=0.1)(x)
    x = tf.keras.layers.Add()([x, residual])
    
    # Second block
    x = SpectralNormalization(tf.keras.layers.Dense(64, activation='relu'))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = DropBlock(drop_rate=0.15)(x)
    x = tf.keras.layers.Dropout(0.4)(x)
    
    # Output layer
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    
    model = tf.keras.Model(inputs, outputs, name='heavily_regularized_model')
    return model

# Test regularization techniques
print("\n=== Testing Advanced Regularization ===")

# Create regularized model
reg_model = create_heavily_regularized_model(X_train.shape[1:], 5)
reg_model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=0.001, weight_decay=0.01),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print(f"Regularized Model Parameters: {reg_model.count_params():,}")

# Train with regularization
reg_history = reg_model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30,
    batch_size=64,
    verbose=0,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(patience=4, factor=0.5)
    ]
)

print(f"Regularized Model - Best Val Acc: {max(reg_history.history['val_accuracy']):.4f}")

# Compare with standard model
standard_model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=X_train.shape[1:]),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

standard_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

standard_history = standard_model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30,
    batch_size=64,
    verbose=0,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=8)]
)

print(f"Standard Model - Best Val Acc: {max(standard_history.history['val_accuracy']):.4f}")

# Compare overfitting
reg_overfit = max(reg_history.history['accuracy']) - max(reg_history.history['val_accuracy'])
std_overfit = max(standard_history.history['accuracy']) - max(standard_history.history['val_accuracy'])

print(f"Regularized Model Overfitting Gap: {reg_overfit:.4f}")
print(f"Standard Model Overfitting Gap: {std_overfit:.4f}")

## 4. Automated Hyperparameter Tuning

In [ ]:
# Keras Tuner integration
try:
    import keras_tuner as kt
    KERAS_TUNER_AVAILABLE = True
except ImportError:
    KERAS_TUNER_AVAILABLE = False
    print("Keras Tuner not available. Implementing basic grid search.")

class BasicHyperparameterTuner:
    """Basic hyperparameter tuning without external dependencies"""
    
    def __init__(self, build_model_fn, objective='val_accuracy'):
        self.build_model_fn = build_model_fn
        self.objective = objective
        self.results = []
        
    def search(self, x_train, y_train, x_val, y_val, param_grid, max_trials=10):
        """Search hyperparameter space"""
        
        print("=== Starting Hyperparameter Search ===")
        
        # Generate parameter combinations
        import itertools
        param_names = list(param_grid.keys())
        param_values = list(param_grid.values())
        combinations = list(itertools.product(*param_values))
        
        # Limit trials
        if len(combinations) > max_trials:
            combinations = np.random.choice(len(combinations), max_trials, replace=False)
            combinations = [combinations[i] for i in range(max_trials)]
        
        for i, params in enumerate(combinations[:max_trials]):
            print(f"\nTrial {i+1}/{min(max_trials, len(combinations))}")
            
            # Create parameter dict
            param_dict = dict(zip(param_names, params))
            print(f"Parameters: {param_dict}")
            
            # Build and train model
            model = self.build_model_fn(**param_dict)
            
            history = model.fit(
                x_train, y_train,
                validation_data=(x_val, y_val),
                epochs=20,
                batch_size=param_dict.get('batch_size', 32),
                verbose=0,
                callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
            )
            
            # Record results
            best_score = max(history.history[self.objective])
            self.results.append({
                'params': param_dict,
                'score': best_score,
                'history': history.history
            })
            
            print(f"Score: {best_score:.4f}")
        
        # Sort by score
        self.results.sort(key=lambda x: x['score'], reverse=True)
        
        print(f"\n=== Search Complete ===")
        print(f"Best score: {self.results[0]['score']:.4f}")
        print(f"Best parameters: {self.results[0]['params']}")
        
        return self.results[0]

# Hyperparameter search space
def build_tunable_model(learning_rate=0.001, units_1=64, units_2=32, 
                       dropout_rate=0.3, batch_norm=True, optimizer_type='adam'):
    """Build model with tunable hyperparameters"""
    
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(units_1, activation='relu', input_shape=(X_train.shape[1],)))
    
    if batch_norm:
        model.add(tf.keras.layers.BatchNormalization())
    
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(units_2, activation='relu'))
    
    if batch_norm:
        model.add(tf.keras.layers.BatchNormalization())
        
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(5, activation='softmax'))
    
    # Choose optimizer
    if optimizer_type == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_type == 'adamw':
        optimizer = tf.keras.optimizers.AdamW(learning_rate=learning_rate, weight_decay=0.01)
    else:
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    
    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Run hyperparameter tuning
def run_hyperparameter_search():
    """Run comprehensive hyperparameter search"""
    
    print("\n=== Hyperparameter Tuning ===")
    
    # Define search space
    param_grid = {
        'learning_rate': [0.001, 0.003, 0.01],
        'units_1': [64, 128, 256],
        'units_2': [32, 64, 128],
        'dropout_rate': [0.2, 0.3, 0.5],
        'batch_norm': [True, False],
        'optimizer_type': ['adam', 'adamw'],
        'batch_size': [32, 64, 128]
    }
    
    # Create tuner
    tuner = BasicHyperparameterTuner(build_tunable_model)
    
    # Run search
    best_trial = tuner.search(
        X_train, y_train, X_test, y_test,
        param_grid, max_trials=15
    )
    
    # Train best model with more epochs
    print("\n=== Training Best Model ===")
    best_model = build_tunable_model(**best_trial['params'])
    
    final_history = best_model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=50,
        batch_size=best_trial['params'].get('batch_size', 64),
        callbacks=[
            tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
            tf.keras.callbacks.ReduceLROnPlateau(patience=5, factor=0.5)
        ]
    )
    
    final_score = max(final_history.history['val_accuracy'])
    print(f"Final optimized model score: {final_score:.4f}")
    
    return best_model, best_trial, final_history

# Run hyperparameter search
if len(X_train) > 1000:  # Only run if we have enough data
    best_model, best_params, final_history = run_hyperparameter_search()
else:
    print("Skipping hyperparameter search for small dataset")

## 5. Performance Monitoring and Visualization

In [ ]:
# Comprehensive training visualization
class TrainingVisualizer:
    """Comprehensive training visualization and analysis"""
    
    def __init__(self):
        self.histories = {}
        
    def add_history(self, name, history):
        """Add training history for comparison"""
        self.histories[name] = history.history if hasattr(history, 'history') else history
        
    def plot_training_curves(self, figsize=(15, 10)):
        """Plot comprehensive training curves"""
        
        n_models = len(self.histories)
        fig, axes = plt.subplots(2, 2, figsize=figsize)
        
        # Colors for different models
        colors = plt.cm.tab10(np.linspace(0, 1, n_models))
        
        # Loss curves
        for i, (name, history) in enumerate(self.histories.items()):
            axes[0, 0].plot(history['loss'], color=colors[i], label=f'{name} (train)', alpha=0.8)
            if 'val_loss' in history:
                axes[0, 0].plot(history['val_loss'], color=colors[i], label=f'{name} (val)', 
                               linestyle='--', alpha=0.8)
        
        axes[0, 0].set_title('Training Loss')
        axes[0, 0].set_xlabel('Epoch')
        axes[0, 0].set_ylabel('Loss')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
        
        # Accuracy curves
        for i, (name, history) in enumerate(self.histories.items()):
            if 'accuracy' in history:
                axes[0, 1].plot(history['accuracy'], color=colors[i], label=f'{name} (train)', alpha=0.8)
            if 'val_accuracy' in history:
                axes[0, 1].plot(history['val_accuracy'], color=colors[i], label=f'{name} (val)', 
                               linestyle='--', alpha=0.8)
        
        axes[0, 1].set_title('Training Accuracy')
        axes[0, 1].set_xlabel('Epoch')
        axes[0, 1].set_ylabel('Accuracy')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)
        
        # Learning rate curves (if available)
        lr_data_available = False
        for i, (name, history) in enumerate(self.histories.items()):
            if 'lr' in history:
                axes[1, 0].plot(history['lr'], color=colors[i], label=f'{name}', alpha=0.8)
                lr_data_available = True
        
        if lr_data_available:
            axes[1, 0].set_title('Learning Rate')
            axes[1, 0].set_xlabel('Epoch')
            axes[1, 0].set_ylabel('Learning Rate')
            axes[1, 0].set_yscale('log')
            axes[1, 0].legend()
            axes[1, 0].grid(True, alpha=0.3)
        else:
            axes[1, 0].text(0.5, 0.5, 'No Learning Rate Data', ha='center', va='center', 
                           transform=axes[1, 0].transAxes, fontsize=14)
            axes[1, 0].set_title('Learning Rate (No Data)')
        
        # Overfitting analysis
        for i, (name, history) in enumerate(self.histories.items()):
            if 'accuracy' in history and 'val_accuracy' in history:
                train_acc = history['accuracy']
                val_acc = history['val_accuracy']
                gap = np.array(train_acc) - np.array(val_acc)
                axes[1, 1].plot(gap, color=colors[i], label=f'{name}', alpha=0.8)
        
        axes[1, 1].set_title('Overfitting Gap (Train - Val Accuracy)')
        axes[1, 1].set_xlabel('Epoch')
        axes[1, 1].set_ylabel('Accuracy Gap')
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)
        axes[1, 1].axhline(y=0, color='black', linestyle='-', alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
    def analyze_training_stability(self):
        """Analyze training stability metrics"""
        
        print("=== Training Stability Analysis ===")
        
        for name, history in self.histories.items():
            print(f"\n{name}:")
            
            if 'val_loss' in history:
                val_loss = np.array(history['val_loss'])
                
                # Loss volatility
                loss_volatility = np.std(np.diff(val_loss))
                print(f"  Validation Loss Volatility: {loss_volatility:.4f}")
                
                # Convergence analysis
                if len(val_loss) >= 10:
                    early_loss = np.mean(val_loss[:5])
                    late_loss = np.mean(val_loss[-5:])
                    improvement = early_loss - late_loss
                    print(f"  Loss Improvement (Early vs Late): {improvement:.4f}")
                
                # Best epoch
                best_epoch = np.argmin(val_loss)
                best_loss = val_loss[best_epoch]
                print(f"  Best Epoch: {best_epoch} (Loss: {best_loss:.4f})")
                
            if 'val_accuracy' in history:
                val_acc = np.array(history['val_accuracy'])
                best_acc = np.max(val_acc)
                best_acc_epoch = np.argmax(val_acc)
                print(f"  Best Accuracy: {best_acc:.4f} at Epoch {best_acc_epoch}")
                
                # Accuracy plateau detection
                if len(val_acc) >= 10:
                    recent_std = np.std(val_acc[-10:])
                    if recent_std < 0.001:
                        print(f"  ⚠️ Accuracy plateau detected (std: {recent_std:.5f})")

# Performance benchmarking
class ModelBenchmark:
    """Comprehensive model benchmarking"""
    
    def __init__(self):
        self.results = {}
        
    def benchmark_model(self, name, model, test_data, num_runs=5):
        """Benchmark model inference performance"""
        
        import time
        
        print(f"\nBenchmarking {name}...")
        
        x_test, y_test = test_data
        
        # Warmup
        for _ in range(3):
            _ = model(x_test[:10], training=False)
        
        # Timing runs
        times = []
        accuracies = []
        
        for run in range(num_runs):
            start_time = time.time()
            predictions = model(x_test, training=False)
            end_time = time.time()
            
            inference_time = end_time - start_time
            times.append(inference_time)
            
            # Calculate accuracy
            if len(predictions.shape) > 1 and predictions.shape[1] > 1:
                pred_classes = tf.argmax(predictions, axis=1)
                accuracy = tf.reduce_mean(tf.cast(tf.equal(pred_classes, y_test), tf.float32))
                accuracies.append(accuracy.numpy())
        
        # Store results
        self.results[name] = {
            'avg_inference_time': np.mean(times),
            'std_inference_time': np.std(times),
            'throughput': len(x_test) / np.mean(times),
            'accuracy': np.mean(accuracies) if accuracies else None,
            'model_size': model.count_params(),
            'memory_usage': self.estimate_memory_usage(model)
        }
        
        print(f"  Avg Inference Time: {self.results[name]['avg_inference_time']:.4f}s")
        print(f"  Throughput: {self.results[name]['throughput']:.1f} samples/sec")
        print(f"  Accuracy: {self.results[name]['accuracy']:.4f}" if accuracies else "  No accuracy computed")
        
    def estimate_memory_usage(self, model):
        """Estimate model memory usage"""
        
        total_params = model.count_params()
        # Assume float32 (4 bytes per parameter)
        memory_mb = (total_params * 4) / (1024 ** 2)
        return memory_mb
        
    def print_benchmark_summary(self):
        """Print comprehensive benchmark summary"""
        
        print("\n=== Model Benchmark Summary ===")
        print(f"{'Model':<20} {'Accuracy':<10} {'Params':<10} {'Memory(MB)':<12} {'Throughput':<15} {'Latency(ms)':<12}")
        print("-" * 95)
        
        for name, result in self.results.items():
            acc_str = f"{result['accuracy']:.4f}" if result['accuracy'] else "N/A"
            latency_ms = result['avg_inference_time'] * 1000
            
            print(f"{name:<20} {acc_str:<10} {result['model_size']:<10,} "
                  f"{result['memory_usage']:<12.1f} {result['throughput']:<15.1f} "
                  f"{latency_ms:<12.2f}")

# Visualize training results
visualizer = TrainingVisualizer()

# Add training histories from previous experiments
if 'callback_history' in locals():
    visualizer.add_history('With_Callbacks', callback_history)

if 'reg_history' in locals():
    visualizer.add_history('Regularized', reg_history)
    
if 'standard_history' in locals():
    visualizer.add_history('Standard', standard_history)

# Plot comparisons
if len(visualizer.histories) > 0:
    visualizer.plot_training_curves()
    visualizer.analyze_training_stability()

# Benchmark models
benchmark = ModelBenchmark()

if 'callback_model' in locals():
    benchmark.benchmark_model('Callback_Model', callback_model, (X_test, y_test))
    
if 'reg_model' in locals():
    benchmark.benchmark_model('Regularized_Model', reg_model, (X_test, y_test))
    
if 'standard_model' in locals():
    benchmark.benchmark_model('Standard_Model', standard_model, (X_test, y_test))

if len(benchmark.results) > 0:
    benchmark.print_benchmark_summary()

## 6. Production Training Pipeline

In [ ]:
# Complete production training pipeline
class ProductionTrainingPipeline:
    """Complete production-ready training pipeline"""
    
    def __init__(self, config):
        self.config = config
        self.model = None
        self.history = None
        self.callbacks = []
        
    def setup_callbacks(self):
        """Setup comprehensive callback system"""
        
        callbacks = []
        
        # Model checkpointing
        callbacks.append(tf.keras.callbacks.ModelCheckpoint(
            filepath=os.path.join(self.config['output_dir'], 'best_model.h5'),
            monitor=self.config['monitor_metric'],
            save_best_only=True,
            mode='max' if 'acc' in self.config['monitor_metric'] else 'min'
        ))
        
        # Early stopping with adaptive patience
        callbacks.append(AdaptiveEarlyStopping(
            monitor=self.config['monitor_metric'],
            patience=self.config['patience'],
            restore_best_weights=True
        ))
        
        # Learning rate scheduling
        if self.config['lr_schedule'] == 'reduce_on_plateau':
            callbacks.append(tf.keras.callbacks.ReduceLROnPlateau(
                monitor=self.config['monitor_metric'],
                factor=0.5,
                patience=self.config['lr_patience'],
                min_lr=1e-7
            ))
        
        # Performance monitoring
        callbacks.append(PerformanceProfiler(self.config['output_dir']))
        
        # TensorBoard logging
        callbacks.append(tf.keras.callbacks.TensorBoard(
            log_dir=os.path.join(self.config['output_dir'], 'tensorboard'),
            histogram_freq=5,
            write_graph=True,
            update_freq='epoch'
        ))
        
        # Custom callbacks
        if self.config.get('gradient_monitoring', False):
            callbacks.append(GradientNormCallback(self.config['output_dir']))
            
        callbacks.append(LearningRateLogger(self.config['output_dir']))
        
        self.callbacks = callbacks
        return callbacks
    
    def create_model(self):
        """Create model based on configuration"""
        
        # Model architecture
        if self.config['model_type'] == 'simple':
            model = tf.keras.Sequential([
                tf.keras.layers.Dense(
                    self.config['units'][0], 
                    activation='relu', 
                    input_shape=(self.config['input_dim'],)
                ),
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.Dropout(self.config['dropout_rate']),
                tf.keras.layers.Dense(self.config['units'][1], activation='relu'),
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.Dropout(self.config['dropout_rate']),
                tf.keras.layers.Dense(self.config['num_classes'], activation='softmax')
            ])
            
        elif self.config['model_type'] == 'regularized':
            model = create_heavily_regularized_model(
                (self.config['input_dim'],), 
                self.config['num_classes']
            )
            
        # Optimizer setup
        if self.config['optimizer'] == 'adamw':
            optimizer = tf.keras.optimizers.AdamW(
                learning_rate=self.config['learning_rate'],
                weight_decay=self.config.get('weight_decay', 0.01)
            )
        else:
            optimizer = tf.keras.optimizers.Adam(
                learning_rate=self.config['learning_rate']
            )
        
        # Compile model
        model.compile(
            optimizer=optimizer,
            loss=self.config['loss_function'],
            metrics=self.config['metrics']
        )
        
        self.model = model
        return model
    
    def train(self, train_data, val_data):
        """Execute training pipeline"""
        
        print("=== Starting Production Training Pipeline ===")
        print(f"Configuration: {self.config}")
        
        # Setup
        self.setup_callbacks()
        model = self.create_model()
        
        print(f"Model created with {model.count_params():,} parameters")
        
        # Training
        start_time = time.time()
        
        self.history = model.fit(
            train_data[0], train_data[1],
            validation_data=val_data,
            epochs=self.config['epochs'],
            batch_size=self.config['batch_size'],
            callbacks=self.callbacks,
            verbose=1
        )
        
        training_time = time.time() - start_time
        
        # Results summary
        best_metric = max(self.history.history[self.config['monitor_metric']])
        
        print(f"\n=== Training Complete ===")
        print(f"Training time: {training_time:.2f} seconds")
        print(f"Best {self.config['monitor_metric']}: {best_metric:.4f}")
        print(f"Total epochs: {len(self.history.history['loss'])}")
        
        # Save final model
        model.save(os.path.join(self.config['output_dir'], 'final_model.h5'))
        
        return model, self.history
    
    def evaluate_and_report(self, test_data):
        """Comprehensive model evaluation and reporting"""
        
        if self.model is None:
            raise ValueError("Model not trained yet. Call train() first.")
            
        print("\n=== Model Evaluation ===")
        
        # Test set evaluation
        test_loss, test_acc = self.model.evaluate(test_data[0], test_data[1], verbose=0)
        print(f"Test Loss: {test_loss:.4f}")
        print(f"Test Accuracy: {test_acc:.4f}")
        
        # Generate predictions
        predictions = self.model.predict(test_data[0])
        pred_classes = np.argmax(predictions, axis=1)
        
        # Classification report
        from sklearn.metrics import classification_report, confusion_matrix
        
        report = classification_report(test_data[1], pred_classes, output_dict=True)
        
        print("\nClassification Report:")
        print(classification_report(test_data[1], pred_classes))
        
        # Save comprehensive report
        final_report = {
            'config': self.config,
            'training_history': {k: [float(x) for x in v] for k, v in self.history.history.items()},
            'test_metrics': {
                'loss': float(test_loss),
                'accuracy': float(test_acc)
            },
            'classification_report': report,
            'model_summary': {
                'total_params': int(self.model.count_params()),
                'trainable_params': int(sum([tf.keras.backend.count_params(w) for w in self.model.trainable_weights]))
            }
        }
        
        report_path = os.path.join(self.config['output_dir'], 'training_report.json')
        with open(report_path, 'w') as f:
            json.dump(final_report, f, indent=2)
            
        print(f"Detailed report saved to: {report_path}")
        
        return final_report

# Production training configuration
production_config = {
    'model_type': 'regularized',
    'input_dim': X_train.shape[1],
    'num_classes': 5,
    'units': [128, 64],
    'dropout_rate': 0.3,
    'optimizer': 'adamw',
    'learning_rate': 0.001,
    'weight_decay': 0.01,
    'loss_function': 'sparse_categorical_crossentropy',
    'metrics': ['accuracy'],
    'epochs': 100,
    'batch_size': 64,
    'monitor_metric': 'val_accuracy',
    'patience': 15,
    'lr_patience': 5,
    'lr_schedule': 'reduce_on_plateau',
    'gradient_monitoring': True,
    'output_dir': log_base_dir
}

# Run production pipeline
pipeline = ProductionTrainingPipeline(production_config)
production_model, production_history = pipeline.train(
    (X_train, y_train), 
    (X_test, y_test)
)

# Generate comprehensive report
final_report = pipeline.evaluate_and_report((X_test, y_test))

print(f"\n🎉 Production training pipeline completed successfully!")
print(f"📊 All artifacts saved to: {log_base_dir}")
print(f"📈 Final test accuracy: {final_report['test_metrics']['accuracy']:.4f}")

## Summary

**File Location:** `notebooks/02_neural_networks_with_keras/06_training_optimization_keras.ipynb`

This comprehensive notebook mastered advanced tf.keras training optimization:

### Advanced Callbacks Implemented:
1. **Adaptive Early Stopping**: Dynamic patience adjustment based on improvement patterns
2. **Learning Rate Logging**: Comprehensive LR tracking with visualization
3. **Gradient Monitoring**: Real-time gradient norm analysis
4. **Performance Profiling**: Memory and timing analysis during training

### Optimization Techniques Covered:
- **Custom Learning Rate Schedules**: Warmup, cosine decay, cyclical, one-cycle policies
- **Advanced Optimizers**: Custom AdamW with gradient clipping and weight decay
- **Hyperparameter Tuning**: Automated search with comprehensive parameter spaces
- **Regularization Methods**: DropBlock, spectral normalization, stochastic depth

### Training Infrastructure:
- **Production Pipeline**: Complete automated training system
- **Comprehensive Monitoring**: Real-time performance tracking
- **Automated Reporting**: Detailed training reports and analysis
- **Benchmark Suite**: Model performance comparison framework

### Key Insights Gained:
- Adaptive callbacks improve training robustness
- Custom learning rate schedules can significantly accelerate convergence
- Comprehensive regularization reduces overfitting effectively
- Automated hyperparameter tuning finds better configurations than manual search
- Production pipelines ensure reproducible, monitored training

### Production Features:
- **Automated Checkpointing**: Best model preservation
- **Performance Monitoring**: Resource usage tracking
- **Comprehensive Logging**: TensorBoard integration with custom metrics
- **Report Generation**: Automated training summary reports

### Next Steps:
- Apply these techniques to computer vision models (CNN architectures)
- Implement distributed training with optimization strategies
- Deploy optimized training pipelines to cloud infrastructure
- Scale hyperparameter tuning to larger search spaces

This optimization toolkit enables building production-ready training systems with state-of-the-art techniques!